# Initialization

## Config

In [1]:
import sympy

FILE_NAME = "EKF"
CYCLE_TIME = 0.01
G_ERR = 1
H_ERR = 10

In [2]:
# Q = sympy.eye(6)
# for i in range(3):
#     Q[i,i]     = G_ERR
#     Q[i+3,i+3] = H_ERR
# R = 0.1*sympy.eye(6)
Q = sympy.MatrixSymbol('Q',6,6)
R = sympy.MatrixSymbol('R',6,6)

In [3]:
# P = 100*Q
P = sympy.MatrixSymbol('P_{k|k}',6,6)
P

P_{k|k}

## Setup

In [4]:
gx,gy,gz,hx,hy,hz = sympy.symbols('g_x g_y g_z h_x h_y h_z')
g = sympy.Matrix([gx,gy,gz])
h = sympy.Matrix([hx,hy,hz])
X = sympy.Matrix([gx,gy,gz,hx,hy,hz])
X

Matrix([
[g_x],
[g_y],
[g_z],
[h_x],
[h_y],
[h_z]])

In [5]:
wx,wy,wz = sympy.symbols('omega_x omega_y omega_z')
w = sympy.Matrix([wx,wy,wz])
ax,ay,az = sympy.symbols('a_x a_y a_z')
a = sympy.Matrix([ax,ay,az])
mx,my,mz = sympy.symbols('m_x m_y m_z')
m = sympy.Matrix([mx,my,mz])

m

Matrix([
[m_x],
[m_y],
[m_z]])

In [6]:
# dg = CYCLE_TIME*w.cross(g)
# dh = CYCLE_TIME*w.cross(h)

F = sympy.Matrix([
    [0, -wz, wy, 0, 0, 0],
    [wz, 0, -wx, 0, 0, 0],
    [-wy, wx, 0, 0, 0, 0],
    [0, 0, 0, 0, -wz, wy],
    [0, 0, 0, wz, 0, -wx],
    [0, 0, 0, -wy, wx, 0],
])

F

Matrix([
[       0, -omega_z,  omega_y,        0,        0,        0],
[ omega_z,        0, -omega_x,        0,        0,        0],
[-omega_y,  omega_x,        0,        0,        0,        0],
[       0,        0,        0,        0, -omega_z,  omega_y],
[       0,        0,        0,  omega_z,        0, -omega_x],
[       0,        0,        0, -omega_y,  omega_x,        0]])

# Predict

Considering that the input $u_k$ is the gyroscope measures $\omega_k$ and the states are the gravity and magnetic north $x = [g,m]^T$

1. State prediction: $\hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1},u) = (T F_k(u_k) + I) \hat{x}_{k-1|k-1}$
1. Covariance prediction: $P_{k|k-1} = F_k(u_k) P_{k-1|k-1} F_k(u_k)^T + Q$

In [7]:
T_F = sympy.symbols('T')*F
X_kp1_est = T_F*X + X
P_kp1_est = T_F*P*T_F.T * Q

In [8]:
X_kp1_est = sympy.simplify(X_kp1_est)
X_kp1_est

Matrix([
[-T*g_y*omega_z + T*g_z*omega_y + g_x],
[ T*g_x*omega_z - T*g_z*omega_x + g_y],
[-T*g_x*omega_y + T*g_y*omega_x + g_z],
[-T*h_y*omega_z + T*h_z*omega_y + h_x],
[ T*h_x*omega_z - T*h_z*omega_x + h_y],
[-T*h_x*omega_y + T*h_y*omega_x + h_z]])

In [9]:
P_kp1_est = sympy.simplify(P_kp1_est)
P_kp1_est

Matrix([
[         0, -T*omega_z,  T*omega_y,          0,          0,          0],
[ T*omega_z,          0, -T*omega_x,          0,          0,          0],
[-T*omega_y,  T*omega_x,          0,          0,          0,          0],
[         0,          0,          0,          0, -T*omega_z,  T*omega_y],
[         0,          0,          0,  T*omega_z,          0, -T*omega_x],
[         0,          0,          0, -T*omega_y,  T*omega_x,          0]])*P_{k|k}*Matrix([
[         0,  T*omega_z, -T*omega_y,          0,          0,          0],
[-T*omega_z,          0,  T*omega_x,          0,          0,          0],
[ T*omega_y, -T*omega_x,          0,          0,          0,          0],
[         0,          0,          0,          0,  T*omega_z, -T*omega_y],
[         0,          0,          0, -T*omega_z,          0,  T*omega_x],
[         0,          0,          0,  T*omega_y, -T*omega_x,          0]])*Q

# Update
1. Measure residual: $ y_k = z - h(\hat{x}_{k|k-1}) $
1. Innovation covariance: $ S_k = H_k P_{k|k-1} H_k^T + R_k $
1. Kalman gain: $ K_k = P_{k|k-1} H_k^T S_k^{-1} $
1. Update state estimate: $ \hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k y_k $
1. Update covariance estimate: $ P_{k|k} = (I - K_k H_k)P_{k|k-1} $

In [10]:
# Update
X = sympy.MatrixSymbol('x_{k|k-1}',6,1)
P = sympy.MatrixSymbol('P_{k|k-1}',6,6)
Z = sympy.MatrixSymbol('z_k',6,1)
hx = X
Y = Z - hx
# S = P+R
# K = P*S.inv()
K = P*sympy.MatrixSymbol('S_inv',6,6)
x_kp1 = X + K*Y
P_kp1 = (sympy.eye(6)- K)*P

In [11]:
x_kp1 = sympy.simplify(x_kp1)
# print(sympy.latex(x_kp1))
x_kp1


P_{k|k-1}*S_inv*(-x_{k|k-1} + z_k) + x_{k|k-1}

In [12]:
P_kp1 = sympy.simplify(P_kp1)
P_kp1

(Matrix([
[1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1]]) - P_{k|k-1}*S_inv)*P_{k|k-1}

# To C code

In [13]:
from sympy.utilities.codegen import codegen
from os import remove

try:
    remove(FILE_NAME+".c")
except FileNotFoundError:
    pass

M = sympy.Matrix([
    sympy.symbols('M[0][0] M[0][1] M[0][2]'),
    sympy.symbols('M[1][0] M[1][1] M[1][2]'),
    sympy.symbols('M[2][0] M[2][1] M[2][2]'),
])

convertions = [
    ('state_estimate', X_kp1_est),
    ('covariance_estimate', P_kp1_est),
    ('state_corrected', x_kp1),
    ('covariance_corrected', P_kp1),
    # ('matrix_inv',sympy.MatrixSymbol('M',6,6).inv()),
    # ('matrix_mul',sympy.MatrixSymbol('M1',6,6)*sympy.MatrixSymbol('M2',6,6)),
    # ('matrix_vect_mul',sympy.MatrixSymbol('M',6,6)*sympy.MatrixSymbol('v',6,1))
    # ('matrix_adj',M.cofactor_matrix())
]

[(c_name, c_code), (h_name, c_header)] = codegen(convertions, "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)